In [ ]:
import lgpio
import numpy as np
import bme280
import smbus2
import csv
import time
import sys
import matplotlib.pyplot as plt

In [ ]:
name= sys.argv[0].split("/")

if(len(name) > 1):
    name= name[-1]


if (len(sys.argv) != 2):
    error=""
    error=error+"Error: wrong number of arguments, execute as: \nsudo env \"PATH=$PATH\" ipython "+name+" csv_name"
    sys.exit(error)

In [ ]:
csv_name= sys.argv[1]+".csv"

port= 1
address= 0x77
bus= smbus2.SMBus(port)
bme280.load_calibration_params(bus,address)

LDR= 10
h= lgpio.gpiochip_open(0)
lgpio.gpio_claim_input(h, LDR)

In [ ]:
measures= [] #array que contiene los valores del sensor

try:
    while True:

        bme280_data = bme280.sample(bus, address)
        humidity= bme280_data.humidity
        pressure= bme280_data.pressure
        temp= bme280_data.temperature
        
        light= lgpio.gpio_read(h, LDR)
        light= 1 - light
        
        measures.append([humidity, pressure, temp, light])
        
        print(humidity, pressure, temp, light)
        time.sleep(1)
except KeyboardInterrupt:
   pass  

In [ ]:
#Crear figura
fig,axs= plt.subplots()
axs.set_title("Sensors measurements")
axs.set_xlabel("Seconds")
axs.set_ylabel("Values")

fig_temp,axs_temp= plt.subplots()
axs_temp.set_title("Temperature")
axs_temp.set_xlabel("Seconds")
axs_temp.set_ylabel("ºC")
axs_temp.set_ylim([0, 30])


hum= []
temp= []
pres= []
light= []
x= []

for i in range(len(measures)):
    hum.append(measures[i][0])
    pres.append(measures[i][1]/10)
    temp.append(measures[i][2])
    light.append((measures[i][3]*50))#multiplico por 50 para verlo mejor en el plot
    x.append(i)
    
axs.plot(x, hum, label="Humidity (%)") 
axs.plot(x, pres, label="Preassure (kPa)")
axs.plot(x, temp, label="Temperature (ºC)")
axs.plot(x, light, label="Light")
axs.legend(loc="upper right")

axs_temp.plot(x, temp)

str_fig="Data_graph_"+sys.argv[1]
fig.savefig(str_fig)
plt.close(fig)

str_temp= "Fig_temp_"+sys.argv[1]
fig_temp.savefig(str_temp)
plt.close(fig_temp)

In [ ]:
data= np.array(measures)
np.savetxt(csv_name,data,delimiter=",",fmt='%s')